In [12]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import re

In [13]:
df = pd.read_excel('questions.xlsx',header=None)

questions = pd.DataFrame(columns=['Question', 'Colour', 'Concept', 'Discipline', 'Place'])
scores = pd.DataFrame(columns=['Question', 'Colour', 'Concept', 'Discipline', 'Place'])

preferences = pd.read_csv('perference_transformed.csv')
user_preferences = preferences.iloc[1-1].to_dict()

decision_datas = pd.read_csv('decision.csv',header=None)
decision_data = decision_datas[decision_datas.iloc[:, 0] == 1]

groups = {
    'Colour': ['PURPLE', 'BLUE', 'GREEN', 'RED', 'ORANGE'],
    'Concept': ['RISK', 'HOPE', 'SAFETY', 'VITALITY', 'POWER'],
    'Discipline': ['LITERATURE', 'PHYSICS', 'MUSIC', 'HISTORY', 'GEOGRAPHY'],
    'Place': ['SEA', 'DESERT', 'CITY', 'MOUNTAIN', 'VILLAGE']
}

value_to_group = {
    value: group
    for group, values in groups.items()
    for value in values
}

for index, row in df.iterrows():
    question_num = row.iloc[0]
    groups = re.findall(r'([A-Z]+) - (\d+), ([A-Z]+) - (\d+), ([A-Z]+) - (\d+), ([A-Z]+) - (\d+)', row.iloc[1])
    if groups:
        groups = groups[0]
        question_row = {
            'Question': question_num,
            'Colour': groups[0],
            'Concept': groups[2],
            'Discipline': groups[4],
            'Place': groups[6]
        }
        score_row = {
            'Question': question_num,
            'Colour': int(groups[1]),
            'Concept': int(groups[3]),
            'Discipline': int(groups[5]),
            'Place': int(groups[7])
        }
        
        questions = pd.concat([questions, pd.DataFrame([question_row])], ignore_index=True)
        scores = pd.concat([scores, pd.DataFrame([score_row])], ignore_index=True)


In [14]:
# Obtain the options and score of the current question
def get_current_options(row, scores, index):
    return {
        'Colour': (row['Colour'], scores.iloc[index]['Colour']),
        'Concept': (row['Concept'], scores.iloc[index]['Concept']),
        'Discipline': (row['Discipline'], scores.iloc[index]['Discipline']),
        'Place': (row['Place'], scores.iloc[index]['Place'])
    }

# record the result
def record_choice(question_num, best_items, current_options, results):
    choices_str = ", ".join(best_items)

    scores = []
    for item in best_items:
        for _, (opt, score) in current_options.items():
            if opt == item:
                scores.append(str(score))
    scores_str = ", ".join(scores)

    result_row = {
        'Question': question_num,
        'Choice': choices_str,
        'Score': scores_str
    }
    results = pd.concat([results, pd.DataFrame([result_row])], ignore_index=True)
    return results

In [15]:
# Q-RL
# Q-Reinforcement Learning
results = pd.DataFrame(columns=['Question', 'Choice', 'Score'])
learning_rate = 0.1
discount_factor = 0.9

# initialize Q values
all_options = set()
for _, row in questions.iterrows():
    all_options.update([row['Colour'], row['Concept'], row['Discipline'], row['Place']])
q_values = {opt: 0 for opt in all_options}

# Q-Learning main loop
for index, row in questions.iterrows():
    current_options = get_current_options(row, scores, index)

    # locate highest Q options (may be multiple)
    options_with_q = [(opt, q_values[opt]) for opt, _ in current_options.values()]
    max_q = max(q for _, q in options_with_q)
    best_items = [opt for opt, q in options_with_q if q == max_q]

    # update Q values
    for opt, s in current_options.values():
        old_q = q_values[opt]
        max_future = max(q_values[o] for o, _ in current_options.values())
        q_values[opt] = old_q + learning_rate * (s + discount_factor * max_future - old_q)

    # record results
    results = record_choice(row['Question'], best_items, current_options, results)

# ensure Score column is comma-separated string
results["Score"] = [",".join(str(x) for x in sc) if isinstance(sc, list) else str(sc) for sc in results["Score"]]

# compute cumulative running average score
running_avg = []
sum_scores, total_cnt = 0, 0
for s in results["Score"]:
    values = [float(x) for x in s.split(",")]
    for v in values:
        sum_scores += v
        total_cnt += 1
    running_avg.append(sum_scores / total_cnt)
results["RunningAvgScore"] = running_avg

plt.figure(figsize=(10, 5))
plt.plot(results["Question"], results["RunningAvgScore"], marker='o')
plt.ylim(2.2, 2.8)
plt.title("Average Score Across Questions (Q-RL)")
plt.xlabel("Question Number")
plt.ylabel("Cumulative Mean Score")
plt.grid(True)
plt.tight_layout()
plt.savefig("average_score/Q_RL_avg_score.png", dpi=400)
plt.close()


In [16]:
results = pd.DataFrame(columns=['Question', 'Choice', 'Score'])
history = {}

for index, row in questions.iterrows():
    current_options = get_current_options(row, scores, index)

    # choose best items according to most recent score history
    if not any(opt in history for opt, _ in current_options.values()):
        best_items = [opt for opt, _ in current_options.values()]
    else:
        items_with_scores = [(opt, history[opt]) for opt, _ in current_options.values() if opt in history]
        max_score = max(sc for _, sc in items_with_scores)
        best_items = [opt for opt, sc in items_with_scores if sc == max_score]

    # update recency memory
    for opt, sc in current_options.values():
        history[opt] = sc

    # record selection + scores
    results = record_choice(row['Question'], best_items, current_options, results)

# convert Score to comma-separated string
results["Score"] = [
    ",".join(str(x) for x in sc) if isinstance(sc, list) else str(sc)
    for sc in results["Score"]
]

# compute cumulative running mean score
running_avg = []
total, count = 0, 0
for s in results["Score"]:
    values = [float(x) for x in s.split(",")]
    for v in values:
        total += v
        count += 1
    running_avg.append(total / count)

results["RunningAvgScore"] = running_avg

# plotting
plt.figure(figsize=(10, 5))
plt.plot(results["Question"], results["RunningAvgScore"], marker='o')
plt.ylim(2.2, 2.8)
plt.title("Average Score Over Questions (R-Item)")
plt.xlabel("Question Number")
plt.ylabel("Cumulative Mean Score")
plt.grid(True)
plt.tight_layout()
plt.savefig("average_score/R_Item_avg_score.png", dpi=400)
plt.close()


In [17]:
results = pd.DataFrame(columns=['Question', 'Choice', 'Score'])
history = {}   # {category: last_score}

# main loop
for index, row in questions.iterrows():
    current_options = get_current_options(row, scores, index)

    # choose categories with highest last score
    if not any(cat in history for cat in current_options.keys()):
        best_categories = list(current_options.keys())
    else:
        cands = [(cat, history[cat]) for cat in current_options.keys() if cat in history]
        max_score = max(sc for _, sc in cands)
        best_categories = [cat for cat, sc in cands if sc == max_score]

    # translate categories → selected items
    best_items = [current_options[cat][0] for cat in best_categories]

    # update recency score memory
    for cat, (item, score) in current_options.items():
        history[cat] = score

    results = record_choice(row['Question'], best_items, current_options, results)

# convert Score to comma-separated string
results["Score"] = [
    ",".join(str(x) for x in sc) if isinstance(sc, list) else str(sc)
    for sc in results["Score"]
]

# compute cumulative running mean score up to each question
running_avg = []
s, n = 0, 0
for score_str in results["Score"]:
    values = [float(x) for x in score_str.split(",")]
    for v in values:
        s += v
        n += 1
    running_avg.append(s / n)

results["RunningAvgScore"] = running_avg

# plotting
plt.figure(figsize=(10, 5))
plt.plot(results["Question"], results["RunningAvgScore"], marker='o')
plt.ylim(2.2, 2.8)
plt.title("Average Score Over Questions (R-Cat)")
plt.xlabel("Question Number")
plt.ylabel("Cumulative Mean Score")
plt.grid(True)
plt.tight_layout()
plt.savefig("average_score/R_Cat_avg_score.png", dpi=400)
plt.close()


In [18]:
results = pd.DataFrame(columns=['Question', 'Choice', 'Score'])
expectations = {}   # {option: (total_score_sum, count)}

# main loop
for index, row in questions.iterrows():
    current_options = get_current_options(row, scores, index)

    # compute expectation for current available options
    exp_list = []
    for option, current_score in current_options.values():
        if option in expectations:
            total, cnt = expectations[option]
            avg = total / cnt
        else:
            avg = 0
        exp_list.append((option, avg))

    # find best items (may have multiple)
    max_avg = max(v for _, v in exp_list)
    best_items = [opt for opt, v in exp_list if v == max_avg]

    # update expectation memory
    for option, current_score in current_options.values():
        if option in expectations:
            total, cnt = expectations[option]
            expectations[option] = (total + current_score, cnt + 1)
        else:
            expectations[option] = (current_score, 1)

    results = record_choice(row['Question'], best_items, current_options, results)

# convert Score to comma-separated string
results["Score"] = [
    ",".join(str(x) for x in sc) if isinstance(sc, list) else str(sc)
    for sc in results["Score"]
]

# compute cumulative running mean score
running_avg = []
s, n = 0, 0
for score_str in results["Score"]:
    values = [float(x) for x in score_str.split(",")]
    for v in values:
        s += v
        n += 1
    running_avg.append(s / n)

results["RunningAvgScore"] = running_avg

# plotting
plt.figure(figsize=(10, 5))
plt.plot(results["Question"], results["RunningAvgScore"], marker='o')
plt.ylim(2.2, 2.8)
plt.title("Average Score Over Questions (EV)")
plt.xlabel("Question Number")
plt.ylabel("Cumulative Mean Score")
plt.grid(True)
plt.tight_layout()
plt.savefig("average_score/EV_avg_score.png", dpi=400)
plt.close()


In [19]:
results = pd.DataFrame(columns=['Question', 'Choice', 'Score'])
score_history = {}   # {option: [scores]}

# main loop
for index, row in questions.iterrows():
    current_options = get_current_options(row, scores, index)

    # compute mean − variance for each option
    score_list = []
    for option, current_score in current_options.values():
        if option in score_history and len(score_history[option]) > 1:
            s = score_history[option]
            mean = sum(s) / len(s)
            std = np.sqrt(sum((x - mean) ** 2 for x in s) / len(s))
            value = mean - std
        else:
            value = 0
        score_list.append((option, value))

    # select items with highest score (may be multiple)
    best_score = max(v for _, v in score_list)
    best_items = [opt for opt, v in score_list if v == best_score]

    # update score memory
    for option, current_score in current_options.values():
        score_history.setdefault(option, []).append(current_score)

    results = record_choice(row['Question'], best_items, current_options, results)

# convert Score list to comma-form string
results["Score"] = [
    ",".join(str(x) for x in sc) if isinstance(sc, list) else str(sc)
    for sc in results["Score"]
]

# compute cumulative running mean score
running_avg = []
total, count = 0, 0
for score_str in results["Score"]:
    vals = [float(x) for x in score_str.split(",")]
    for v in vals:
        total += v
        count += 1
    running_avg.append(total / count)

results["RunningAvgScore"] = running_avg

# plotting
plt.figure(figsize=(10, 5))
plt.plot(results["Question"], results["RunningAvgScore"], marker='o')
plt.ylim(2.2, 2.8)
plt.title("Average Score Over Questions (RA)")
plt.xlabel("Question Number")
plt.ylabel("Cumulative Mean Score")
plt.grid(True)
plt.tight_layout()
plt.savefig("average_score/RA_score.png", dpi=400)
plt.close()


In [20]:
results = pd.DataFrame(columns=['Question', 'Choice', 'Score'])
score_history = {}   # {option: [scores]}

# main loop
for index, row in questions.iterrows():
    current_options = get_current_options(row, scores, index)

    # compute mean − variance for each option
    score_list = []
    for option, current_score in current_options.values():
        if option in score_history and len(score_history[option]) > 1:
            s = score_history[option]
            mean = sum(s) / len(s)
            std = np.sqrt(sum((x + mean) ** 2 for x in s) / len(s))
            value = mean - std
        else:
            value = 0
        score_list.append((option, value))

    # select items with highest score (may be multiple)
    best_score = max(v for _, v in score_list)
    best_items = [opt for opt, v in score_list if v == best_score]

    # update score memory
    for option, current_score in current_options.values():
        score_history.setdefault(option, []).append(current_score)

    results = record_choice(row['Question'], best_items, current_options, results)

# convert Score list to comma-form string
results["Score"] = [
    ",".join(str(x) for x in sc) if isinstance(sc, list) else str(sc)
    for sc in results["Score"]
]

# compute cumulative running mean score
running_avg = []
total, count = 0, 0
for score_str in results["Score"]:
    vals = [float(x) for x in score_str.split(",")]
    for v in vals:
        total += v
        count += 1
    running_avg.append(total / count)

results["RunningAvgScore"] = running_avg

# plotting
plt.figure(figsize=(10, 5))
plt.plot(results["Question"], results["RunningAvgScore"], marker='o')
plt.ylim(2.2, 2.8)
plt.title("Average Score Over Questions (RS)")
plt.xlabel("Question Number")
plt.ylabel("Cumulative Mean Score")
plt.grid(True)
plt.tight_layout()
plt.savefig("average_score/RS_avg_score.png", dpi=400)
plt.close()